In [ ]:
def is_main_module():
    """
    Returns whether this notebook is the main module
    i.e. not being run from anotherr notebook
    """
    return __name__ == '__main__' and '__file__' not in globals()

SAVE_FIGURES = False
SAVE_FIGURE_DPI = 300
DEBUG_PRINT_PLOTS = True
SAVE_FIGURE_FORMAT = "PDF"  # PDF, SVG, PNG, for saved images

In [ ]:
# Preamble

#import libraries
import sys
# add the path to the Python module directory
sys.path.insert(0, '../Python')

import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
pd.set_option("display.precision", 4)

import Constants
import Utils

import SphericalEarthModel as SEM
import Prf as PRF
import Nesz as NESZ
import Aasr as AASR
import Rasr as RASR

import rectangularantenna as RA
import lineararrayantenna as LAA

def green(val):
    color = 'lightgreen'
    return 'background-color: %s' % color

def yellow(val):
    color = 'yellow'
    return 'background-color: %s' % color

def red(val):
    color = 'red'
    return 'background-color: %s' % color

def blue(val):
    color = 'lightblue'
    return 'background-color: %s' % color

In [ ]:
%matplotlib inline
%config Inlinebackend.figure_format = 'svg'

# Reflector Beamforming Verification

## 10m C-band Dish Stripmap Single-Pol Mode (Case 1)

Utilises 26 RX elements in elevation

Total swath covered: 120 km

Incidence Angle coverage: 32.07 - 40.6 degrees

Look Angle coverage: 28.8 - 36.1 degrees

Nominal range resolution: 6 m

Nominal azimuth resolution: 6 m

Antenna mounting angle (antenna normal to the nadir direction): 32.45 degrees

NESZ: -25 dB

Making use of SCORE by selecting the appropriate RX channel on reception. 

The receiving beam is scanned across the entire swath (SCORE).

In [ ]:
# system parameters
Frequency                 = 5.4e9
Wavelength                = Constants.SPEED_OF_LIGHT / Frequency
Altitude                  = 650e3
Prf                       = 1610
TxDutyCycle               = 0.08
PulseLen                  = (1/Prf) * TxDutyCycle
AntennaOffset             = 32.45     # Antenna Mounting angle/Mid-Look angle in degrees
AntennaDiameter           = 10.       # Tandem-L antenna diameter
Losses                    = 3.6       # System losses: Processing atmosphere, taper, degradation, etc. in dB
ProcessedAzimuthBandwidth = 1150      # Doppler bandwidth in Hz
NumElChannels             = 26        # Number of feeds in elevation per azimuth channel
NumAzChannels             = 1         # Number of azimuth channels. Each consisting of NumElChannels in elevation
NumRxBeams                = 26

# System requirements
ReqNesz                   = -25       # Required NESZ in dB
ReqRasr                   = -20       # Required RASR in dB
ReqAasr                   = -20       # Required AASR in dB
ReqResolutionAz           = 6         # Required Azimuth resolution in meters   (cross-range) 
ReqResolutionRg           = 6         # Required Elevation resolution in meters (ground range)
ReqGroundSwath            = 125e3     # Required ground swath in meters

# Derived Requirements
TxPowerAvg                = 300       # Average transmitted power
TxPowerPeakPerTRM         = (TxPowerAvg / TxDutyCycle) / (NumElChannels * NumAzChannels) # Peak transmit power per Transmit/Receive Module (TRM)
MidSwathIncidenceAngle    = SEM.IncidenceAngle(Altitude, Utils.deg2rad(AntennaOffset))
ChirpBandwidth            = Constants.SPEED_OF_LIGHT / (2*ReqResolutionRg*np.sin(MidSwathIncidenceAngle)) # pulse bandwidth
#NoiseTemperature          = 649
#NoiseFigure               = (NoiseTemperature / Constants.STANDARD_TEMPERATURE) + 1
NoiseFigure               = 3

print("Altitude:               {:.2f} km".format(Altitude / 1000))
print("Center Frequency:       {:.2f} GHz".format(Frequency / 1e9))
print("Wavelength:             {:.3f} m".format(Wavelength))
print("Num Elevation Channels: {}".format(NumElChannels))
print("Num Azimuth Channels:   {}".format(NumAzChannels))
print("Average TX Power:       {:.1f} W".format(TxPowerAvg))
print("Peak (TRM) TX Power:    {:.1f} W".format(TxPowerPeakPerTRM))
print("Average PRF:            {:.1f} Hz".format(Prf))
print("Pulse length:           {:.2f} us".format(PulseLen * 1e6))
print("Noise Figure:           {:.2f} dB".format(NoiseFigure))
print("Chirp Bandwidth:        {:.2f} MHz".format(ChirpBandwidth/1e6))

### Geometry

In [ ]:
# geometry code
LookAngleRange = np.arange(28.8, 36.2, 0.1)
LookAngleRangeRad = LookAngleRange * (np.pi/180)
print("Look angle ranges:      {:.2f} - {:.2f} deg".format(LookAngleRange.min(), LookAngleRange.max()))

IncidenceAngleRangeRad = SEM.IncidenceAngle(Altitude, LookAngleRangeRad)
IncidenceAngleRange = IncidenceAngleRangeRad * (180/np.pi)
print("Incidence angle ranges: {:.2f} - {:.2f} deg".format(IncidenceAngleRange.min(), IncidenceAngleRange.max()))

#### Swath analysis

In [ ]:
# Get the ground swath width for the different look angles and TX beamwidths
Test_Beamwidths = np.arange(1, 61, 1)          # beamwidth ranges between 1 and 60 degrees
Test_LookAngles = np.arange(20, 45., 5)
Test_LookAngles = np.append(Test_LookAngles, AntennaOffset)
Test_Altitudes = Altitude

Test_wgr = []
for la in Test_LookAngles:
    _, _wgr = SEM.SwathGroundSwathFromBeamwidth(Test_Altitudes, Utils.deg2rad(la), Utils.deg2rad(Test_Beamwidths))
    Test_wgr.append(_wgr)
Test_wgr = np.asarray(Test_wgr)

# Plot the ground swath widths vs beamwidths & look angles
fig, ax1 = plt.subplots(1,1,figsize = (10,6))
for i in range(Test_wgr.shape[0]):
    ax1.plot(Test_Beamwidths, Test_wgr[i,:] / 1000, label=r"Look Angle = %.2f deg" % (Test_LookAngles[i]))
ax1.legend()
ax1.set_title('Ground Swath Width vs Elevation Beamwidth \n Altitude = %d km' % (Test_Altitudes / 1000))
ax1.set_xlabel(r"Elevation Beamwidth (deg)")
ax1.set_ylabel(r"Ground swath Width (km)")
ax1.grid(linestyle='dotted')
ax1.set_ylim(10, 440)
ax1.set_xlim(0,30)
# TX beam
ax1.axvline(x=7.3, linestyle="dotted")
ax1.axhline(y=ReqGroundSwath/1000, linestyle="dotted")

# RX beam
ax1.axvline(x=4.375, linestyle="dotted", color="tab:green")
ax1.axhline(y=87, linestyle="dotted", color="tab:green")

plt.tight_layout()
plt.show()

# TX Beam
SetTxBeamwidth = 7.3
sw, gw =  SEM.SwathGroundSwathFromBeamwidth(Test_Altitudes, Utils.deg2rad(AntennaOffset), Utils.deg2rad(SetTxBeamwidth))
print("TX Swath  = {:.2f} km and Ground swath = {:.2f} km @ Look angle = {:.2f} deg and Elevation BW = {:.2f} deg".format(sw/1000, gw/1000, AntennaOffset, SetTxBeamwidth))

In [ ]:
# Look angle and incidence angle calculations for subswaths
initial_look_angle = 28.8
# initial_beam_width = 4.375
initial_beam_width_Deg = (LookAngleRange.max() - LookAngleRange.min()) / (NumRxBeams-1)
print("RX single feed beamwidth: {:.3f} deg".format(initial_beam_width_Deg))

Beam_look_angle_Deg = []
for i in range(NumRxBeams):
     Beam_look_angle_Deg.append(initial_look_angle + i*initial_beam_width_Deg)

Beam_look_angle_Deg = np.asarray(Beam_look_angle_Deg)
Beam_look_angle = Utils.deg2rad(Beam_look_angle_Deg)

Beam_incidence_angle = SEM.IncidenceAngle(Altitude, Beam_look_angle)
Beam_incidence_angle_Deg = Utils.rad2deg(Beam_incidence_angle)

antenna_normal_incidence_angle = SEM.IncidenceAngle(Altitude, Utils.deg2rad(AntennaOffset))

# for i in range(NumRxBeams):
#     print("RX Beam {}:\tIncidence Angle: {:.2f}\tLook Angle: {:.2f} deg".format(i,
#                                                                                Beam_incidence_angle_Deg[i],
#                                                                                Utils.rad2deg(Beam_look_angle[i])))

### TX Antenna

In [ ]:
# TX Antenna code
TX_efficiency   = 0.6    #0.5144
TX_BW           = SetTxBeamwidth   # degrees

# the antenna elevation dimension is estimated from the beamwidth required to illuminate the entire swath
TX_elv_dim = RA.RectangularAntenna.dimension_for_beamwidth(Wavelength, Utils.deg2rad(TX_BW), TX_efficiency)

# initialise the TX object
tandemL_TX = RA.RectangularAntenna(length= AntennaDiameter, height= TX_elv_dim, efficiency= TX_efficiency)

TX_swath, TX_gswath =  SEM.SwathGroundSwathFromBeamwidth(Altitude, Utils.deg2rad(AntennaOffset), Utils.deg2rad(TX_BW))

TX_gain = tandemL_TX.gain(Wavelength)

# --------------------------------------------------------
# display the RX antenna characteristics in a table format
df = pd.DataFrame (
    {
        "Parameter" : ['TX Length (Azimuth)', 'TX Height (Elevation)', 
                       'TX Azimuth Beamwidth', 'TX Elevation Beamwidth', 
                       'TX Gain', 'Ground Swath'],
        "Value" : [tandemL_TX.length(), tandemL_TX.height(),
                   Utils.rad2deg(tandemL_TX.azimuth_beamwidth(Wavelength)), 
                   Utils.rad2deg(tandemL_TX.elevation_beamwidth(Wavelength)),
                   10*np.log10(tandemL_TX.gain(Wavelength)), TX_gswath / 1000],
        "Unit" : ['m', 'm', 
                  'deg', 'deg',
                  'dBi', 'km']
    }
)
titles = ['Parameter', 'Value', 'Unit']
df.reindex(columns=titles) \
  .style.applymap(yellow, subset=pd.IndexSlice[1:1, :]) \
        .applymap(yellow, subset=pd.IndexSlice[3:4, :]) \
        .applymap(green, subset=pd.IndexSlice[5:5, :])

### RX Antenna Beams

For the receive beams, the entire reflector surface is illuminated, but the beam is focused on individual elements using SCORE-based DBF.

In [ ]:
# initialise the RX antenna object
RX_efficiency      = 0.6    # 0.5144
RX_BW_sub_aperture = initial_beam_width_Deg  # degrees

# the antenna diameter is estimated from the beamwidth
RX_diameter_sub_aperture = (RX_efficiency * Wavelength) / Utils.deg2rad(RX_BW_sub_aperture)

# the gain however is determined by the full surface of the reflector
Area_full = math.pi * (AntennaDiameter/2)**2
RX_gain_full = (4*math.pi*RX_efficiency * Area_full) / (math.pow(Wavelength, 2))

tandemL_RX_Beams = []
for i in range(NumRxBeams):
    tandemL_RX_Beams.append(RA.RectangularAntenna(AntennaDiameter, RX_diameter_sub_aperture, RX_efficiency))

RX_gain_sub_apertures = tandemL_RX_Beams[0].gain(Wavelength)

# swath covered by RX beam
RX_ground_swaths = []
for i in range(NumRxBeams):
    _, rxGroundSwath = SEM.SwathGroundSwathFromBeamwidth(Altitude, Beam_look_angle[i], Utils.deg2rad(RX_BW_sub_aperture))
    RX_ground_swaths.append(rxGroundSwath)
    
# --------------------------------------------------------------------
# display the TX antenna characteristics in a table format
pd.set_option("display.precision", 3)
df = pd.DataFrame (
    {
        "Parameter" : ['RX Height (Elevation)', 'RX Length (Azimuth)', 'Efficiency',
                       'RX Elevation Beamwidth', 'RX Azimuth Beamwidth',
                       'RX Gain (full surface)'],
        "Value" : [RX_diameter_sub_aperture, AntennaDiameter, RX_efficiency,
                   Utils.rad2deg(tandemL_RX_Beams[0].elevation_beamwidth(Wavelength)), 
                   Utils.rad2deg(tandemL_RX_Beams[0].azimuth_beamwidth(Wavelength)),
                   10*np.log10(RX_gain_full)],
        "Unit" : ['m','m', '', 
                  'deg','deg','dBi']
    }
)
titles = ['Parameter', 'Value', 'Unit']
df.reindex(columns=titles) \
  .style.applymap(yellow, subset=pd.IndexSlice[1:1, :]) \
        .applymap(yellow, subset=pd.IndexSlice[3:3, :]) \
        .applymap(yellow, subset=pd.IndexSlice[5:5, :]) \
        .applymap(green, subset=pd.IndexSlice[6:9, :])

### Antenna Patterns

In [ ]:
# antenna pattern for TX and RX antennas
AntennaPatternRange = np.arange(-90, 90, 0.1)

# Azimuth angles used only to visualise the antenna pattern in azimuth
azimuthAngles = np.arange(-5, 5, 0.01)   # azimuth angles in degrees

# ---------------------------------------------------
# TX antenna power pattern
txElvPat = tandemL_TX.elevation_power_pattern(Wavelength, Utils.deg2rad(AntennaPatternRange), 0)
txAzPat = tandemL_TX.azimuth_power_pattern(Wavelength, Utils.deg2rad(azimuthAngles), 0)

# RX antenna power patterns
rxElvPat = []
rxAzPat = []
for i in range(NumRxBeams):
    _rxElvPat = tandemL_RX_Beams[i].elevation_power_pattern(Wavelength, Utils.deg2rad(AntennaPatternRange), 0)
    _rxAzPat = tandemL_RX_Beams[i].azimuth_power_pattern(Wavelength, Utils.deg2rad(azimuthAngles), 0)
    rxElvPat.append(_rxElvPat)
    rxAzPat.append(_rxAzPat)

# TX / RX antenna gain scaling factor
gainScales = []
for i in range(NumRxBeams):
    #gainScales.append(tandemL_TX.gain(Wavelength) / tandemL_RX_Beams[i].gain(Wavelength))
    # use the entire reflector surface for the RX gain
    gainScales.append(tandemL_TX.gain(Wavelength) / RX_gain_full)

# ---------------------------------------------------
# Plot the antenna patterns
fig, (ax1, ax2) = plt.subplots(2,1, figsize = (12,6))
# TX pattern
ax1.plot(AntennaPatternRange, 10*np.log10(txElvPat * gainScales[0]), color="blue", label=r"TX")                  
# rx patterns
for i in range(NumRxBeams):
    RX_BW = Utils.rad2deg(tandemL_RX_Beams[i].elevation_beamwidth(Wavelength))
    if i == 0:
        ax1.plot(AntennaPatternRange + (-((NumRxBeams-1)/2)*RX_BW) + (i*RX_BW), 10*np.log10(rxElvPat[i]), linestyle="dashed", label=r"RX")
    else:
        ax1.plot(AntennaPatternRange + (-((NumRxBeams-1)/2)*RX_BW) + (i*RX_BW), 10*np.log10(rxElvPat[i]), linestyle="dashed")

ax1.legend(loc=2)
ax1.set_title(r"Normalised Planar Array Elevation Antenna Pattern, $G_{el}(\theta)$ vs Angle")
ax1.set_xlabel(r"Angle from Antenna Center (deg)")
ax1.set_ylabel(r"Antenna pattern, $G_{el}(\theta)$ (dB)")
ax1.grid(linestyle='dotted')
ax1.set_ylim(-40, 5)
ax1.set_xlim(-25, 25)

# azimuth patterns
# TX pattern
ax2.plot(azimuthAngles, 10*np.log10(txAzPat * gainScales[0]), color="blue", label=r"TX")
# RX patterns
for i in range(NumRxBeams):
    if i == 0:
        ax2.plot(azimuthAngles, 10*np.log10(rxAzPat[i]), linestyle="dashed", color="green", label=r"RX")
    else:
        ax2.plot(azimuthAngles, 10*np.log10(rxAzPat[i]), linestyle="dashed", color="green")

ax2.legend(loc=2)
ax2.set_title(r"Normalised Planar Array Azimuth Power Pattern, $G_{az}(\theta)$ vs Angle")
ax2.set_xlabel(r"Angle from Antenna Center (deg)")
ax2.set_ylabel(r"Antenna pattern, $G_{az}(\theta)$ (dB)")
ax2.grid(linestyle='dotted')
ax2.set_ylim(-40, 5)
ax2.set_xlim(azimuthAngles.min(), azimuthAngles.max())

plt.tight_layout()
plt.show()

### PRF Analysis

In [ ]:
antLength = AntennaDiameter

#### PRF vs Altitude

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 30
    numNadirEchoes = 11
    
    # define the swath width (in meters) ranges to use 
    altitudes = np.arange(450e3, 805e3, 5e3)
    
    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, altitudes)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(altitudes)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(altitudes, MidSwathIncidenceAngle)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(ReqGroundSwath,
                                             altitudes, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadir limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(altitudes/1000, nadirLower, color="red", alpha=0.6)
        ax1.plot(altitudes/1000, nadirUpper, color="red", alpha=0.6)
        plt.fill_between(altitudes/1000, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(altitudes/1000, prfLower, color="black", alpha=0.6)
        ax1.plot(altitudes/1000, prfUpper, color="black", alpha=0.6)
        plt.fill_between(altitudes/1000, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.plot(altitudes/1000, prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(altitudes/1000, prfMax, color='tab:orange', linestyle='dashed') 
    # plot the altitude and ideal PRF
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    ax1.axvline(x=Altitude/1000, color="black", linestyle="dashed")
    
    #ax1.set_title('PRF vs Altitude')
    ax1.set_xlabel(r'Altitude (km)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(1400, 1800)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_15a_refl_verification_01_prf_vs_alt.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_15a_refl_verification_01_prf_vs_alt.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_15a_refl_verification_01_prf_vs_alt.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

#### PRF vs Antenna Length

In [ ]:
# define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses   = 32
    numNadirEchoes = 12
    
    # define the antenna length range to cover
    antennaLength = np.arange(6, 16.1, 0.1)
    
    # get the minimum PRF
    prfMin = []
    for ant in (antennaLength):
        prfMin.append(PRF.PrfMin(ant, Altitude))
    prfMin = np.asarray(prfMin)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(Altitude, MidSwathIncidenceAngle)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(ReqGroundSwath,
                                             Altitude, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay:            {:.6f} s".format(nadirDelay))
    print("Near range Delay (tn):  {:.6f} s".format(nearDelay))
    print("Far range Delay (tf):   {:.6f} s".format(farDelay))
    print("tf - tn:                {:.6f} s".format(farDelay - nearDelay))
    print("PRF max:                {:.6f} Hz".format(prfMax))  
    print("PRF min ({:.3}m antenna): {:.6f} Hz".format(antLength, PRF.PrfMin(antLength, Altitude)))  
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.axhline(y=nadirLower, color="red", alpha=0.6)
        ax1.axhline(y=nadirUpper, color="red", alpha=0.6)
        plt.fill_between(antennaLength, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.axhline(y=prfLower, color="black", alpha=0.6)
        ax1.axhline(y=prfUpper, color="black", alpha=0.6)
        plt.fill_between(antennaLength, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.plot(antennaLength, prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.axhline(y=prfMax, color='tab:orange', linestyle='dashed') 
    # plot antenna length and ideal PRF
    ax1.axhline(y=Prf, color="black", linestyle='dashed') 
    ax1.axvline(x=antLength, color='black', linestyle='dashed') 
    
    
    #ax1.set_title('PRF vs Antenna length')
    ax1.set_xlabel(r'Antenna length (m)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(1400, 1800)
    plt.grid(linestyle='dotted')
    ax1.margins(0)
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_15b_refl_verification_01_prf_vs_ant.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_15b_refl_verification_01_prf_vs_ant.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_15b_refl_verification_01_prf_vs_ant.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

#### PRF vs Pulse Length

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 30
    numNadirEchoes = 11
    
    # define the pulse length range to cover
    pulseLengths = np.arange(0, 101e-6, 1e-6)
    
    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(Altitude, MidSwathIncidenceAngle)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(ReqGroundSwath,
                                             Altitude, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             pulseLengths)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       pulseLengths,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(pulseLengths, (farDelay - nearDelay))
    
    # display/plot the results
    print("Near range delay: {:.6f} s".format(nearDelay))
    print("Far range delay:  {:.6f} s".format(farDelay))
    print("Nadir delay:      {:.6f} s".format(nadirDelay))
    print("Minimum PRF:      {:.2f} Hz".format(prfMin))
    
    # plot PRF vs pulse length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(pulseLengths* 1e6, nadirLower, color="red", alpha=0.6)
        ax1.plot(pulseLengths* 1e6, nadirUpper, color="red", alpha=0.6)
        plt.fill_between(pulseLengths* 1e6, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(pulseLengths* 1e6, prfLower, color="black", alpha=0.6)
        ax1.plot(pulseLengths* 1e6, prfUpper, color="black", alpha=0.6)
        plt.fill_between(pulseLengths* 1e6, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(pulseLengths* 1e6, prfMax, color='tab:orange', linestyle='dashed')  
    # plot pulse length and ideal PRF
    ax1.axvline(x=PulseLen*1e6, color='black', linestyle='dashed')  
    ax1.axhline(y=Prf, color='black', linestyle='dashed')  
    
    #ax1.set_title('PRF vs Pulse Lengh')
    ax1.set_xlabel(r'Pulse Lengh, $\tau_p$ ($\mu$s)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(1400, 1800)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_15c_refl_verification_01_prf_vs_pul.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_15c_refl_verification_01_prf_vs_pul.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_15c_refl_verification_01_prf_vs_pul.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

#### PRF vs Swath Width

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 30
    numNadirEchoes = 11
    
    # define the swath width (in meters) ranges to use 
    groundSwaths = np.arange(5e3, 201e3, 1e3)
    
    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = Utils.deg2rad(AntennaOffset)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(groundSwaths,
                                             Altitude, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay: {:.6f} s".format(nadirDelay))
    print("Minimum PRF: {:.2f} Hz".format(prfMin))
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(groundSwaths/1000, nadirLower, color="red", alpha=0.6)
        ax1.plot(groundSwaths/1000, nadirUpper, color="red", alpha=0.6)
        plt.fill_between(groundSwaths/1000, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(groundSwaths/1000, prfLower, color="black", alpha=0.6)
        ax1.plot(groundSwaths/1000, prfUpper, color="black", alpha=0.6)
        plt.fill_between(groundSwaths/1000, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(groundSwaths/1000, prfMax, color='tab:orange', linestyle='dashed')     
    
    # plot ideal ground swath and prf
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    ax1.axvline(x=ReqGroundSwath/1000, color="black", linestyle="dashed")
    
    #ax1.set_title('PRF vs Ground Swath')
    ax1.set_xlabel(r'Ground swath, $W_{rg}$ (km)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(1400, 1800)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    #plt.show()
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_15d_refl_verification_01_prf_vs_swa.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_15d_refl_verification_01_prf_vs_swa.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_15d_refl_verification_01_prf_vs_swa.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

#### PRF vs Incidence Angle

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 41
    numNadirEchoes = 25
    
    # define the incidence angle (in degreees) ranges to use 
    incidenceAngles = Utils.deg2rad(np.arange(35, 38, 1))

    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(Altitude, incidenceAngles)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(ReqGroundSwath,
                                             Altitude, 
                                             incidenceAngles,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay: {:.6f} s".format(nadirDelay))
    print("Minimum PRF: {:.2f} Hz".format(prfMin))
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(Utils.rad2deg(incidenceAngles), prfLower, color="black", alpha=0.6)
        ax1.plot(Utils.rad2deg(incidenceAngles), prfUpper, color="black", alpha=0.6)
        plt.fill_between(Utils.rad2deg(incidenceAngles), prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(Utils.rad2deg(incidenceAngles), nadirLower, color="red", alpha=0.6)
        ax1.plot(Utils.rad2deg(incidenceAngles), nadirUpper, color="red", alpha=0.6)
        plt.fill_between(Utils.rad2deg(incidenceAngles), nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(Utils.rad2deg(incidenceAngles), prfMax, color='tab:orange', linestyle='dashed')     
    # plot the incidence angle and ideal PRF
    ax1.axvline(x=Utils.rad2deg(MidSwathIncidenceAngle), color="black", linestyle="dashed")
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    
    #ax1.set_title('PRF vs Incidence Angle')
    ax1.set_xlabel(r'Incidence Angle, $\theta_{i,m}$ (deg)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(1400, 1800)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    #plt.show()    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_15e_refl_verification_01_prf_vs_inc.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_15e_refl_verification_01_prf_vs_inc.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_15e_refl_verification_01_prf_vs_inc.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

#### PRF vs Look Angle

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 53
    numNadirEchoes = 36
    
    # define the look angle (in degreees) ranges to use 
    lookAngles = Utils.deg2rad(np.arange(32, 33.1, 0.1))
    
    # calculate the incidence angles from these look angles
    #incidenceAngles = IncidenceAngleRangeRad
    incidenceAngles = SEM.IncidenceAngle(Altitude, lookAngles)

    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(ReqGroundSwath,
                                             Altitude, 
                                             incidenceAngles,
                                             lookAngles)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay: {:.2f} us".format(nadirDelay * 1e6))
    print("Minimum PRF: {:.2f} Hz".format(prfMin))
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))   
    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(Utils.rad2deg(lookAngles), prfLower, color="black", alpha=0.6)
        ax1.plot(Utils.rad2deg(lookAngles), prfUpper, color="black", alpha=0.6)
        plt.fill_between(Utils.rad2deg(lookAngles), prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(Utils.rad2deg(lookAngles), nadirLower, color="red", alpha=0.6)
        ax1.plot(Utils.rad2deg(lookAngles), nadirUpper, color="red", alpha=0.6)
        plt.fill_between(Utils.rad2deg(lookAngles), nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRF Maximum
    ax1.plot(Utils.rad2deg(lookAngles), prfMax, color='tab:orange', linestyle='dashed')   
    # plot antenna mounting angle and ideal prf
    ax1.axvline(x=AntennaOffset, color="black", linestyle="dashed")
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    
    #ax1.set_title('PRF vs Look Angle')
    ax1.set_xlabel(r'Look Angle, $\gamma_m$ (deg)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(1400, 1800)
#     ax1.set_xlim(13, 46)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    #plt.show()
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_15f_refl_verification_01_prf_vs_loo.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_15f_refl_verification_01_prf_vs_loo.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_15f_refl_verification_01_prf_vs_loo.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

### Performance

#### NESZ

In [ ]:
# Calculate the antenna patterns used for calculating the NESZ
NumSamples = 100
NumRxScanAngles = NumRxBeams
NumTxScanAngles = 1

txAnt = tandemL_TX
rxAnt = tandemL_RX_Beams[0]

RxAntPatterns, TxAntPatterns, \
rxtxAnglesRad, rxPeaks, \
RxAntPatMaxAngles, RxAntPatMaxValues \
    = NESZ.CalNeszAntennaPatterns_rect(
        rxAnt, 
        txAnt, 
        Wavelength, 
        NumRxScanAngles, 
        NumTxScanAngles,
        LookAngleRangeRad.min(), 
        LookAngleRangeRad.max(),
        NumSamples)
print(Utils.rad2deg(LookAngleRangeRad.min()), Utils.rad2deg(LookAngleRangeRad.max()))
print(AntennaOffset)
# -----------------------------------------------
# Plot the antenna patterns
if DEBUG_PRINT_PLOTS:
    fig, ax1 = plt.subplots(figsize = (8,3))
    for i in range(NumRxScanAngles * NumTxScanAngles):
        if i == 0:
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + Utils.deg2rad(AntennaOffset)), 10*np.log10(TxAntPatterns[i]), color="blue", label=r"TX Elevation Pattern")
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + Utils.deg2rad(AntennaOffset)), 10*np.log10(RxAntPatterns[i]), color="red", linestyle="dashdot", label=r"RX Elevation Pattern")
        else:
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + Utils.deg2rad(AntennaOffset)), 10*np.log10(TxAntPatterns[i]), color="blue")
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + Utils.deg2rad(AntennaOffset)), 10*np.log10(RxAntPatterns[i]), color="red", linestyle="dashdot")

    ax1.plot(Utils.rad2deg(RxAntPatMaxAngles/1000  + Utils.deg2rad(AntennaOffset)), 10*np.log10(RxAntPatMaxValues), color="green", label=r"Effective RX Elevation Pattern")

    ax1.set_title("3dB Antenna pattern vs Look Angle")
    ax1.set_xlabel(r"Look Angle (deg)")
    ax1.set_ylabel(r"Antenna pattern (dB)")
    ax1.grid(linestyle='dotted')
    ax1.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# calculate the incidence angles and slant ranges over the antenna beam pattern
IA_Arr, SR_Arr = NESZ.CalcAnglesAndSlantRanges(
                    NumRxScanAngles * NumTxScanAngles, 
                    Altitude, 
                    rxtxAnglesRad, 
                    Utils.deg2rad(AntennaOffset))

In [ ]:
# calculate the core angles subtended by the slant ranges, and from these the ground ranges
GR_Arr = NESZ.CalcGroundRange(
            NumRxScanAngles * NumTxScanAngles, 
            Altitude, 
            SR_Arr)

In [ ]:
# Calulate the value of NESZ of the look angles
NESZ_Arr, Nesz_max_incidence_angles, Nesz_max_values \
    = NESZ.CalcNesz(
            NumRxScanAngles * NumTxScanAngles, 
            Altitude, 
            ChirpBandwidth, 
            IA_Arr, SR_Arr, 
            TxAntPatterns, RxAntPatterns, 
            Wavelength, TxPowerPeakPerTRM * (NumElChannels * NumAzChannels), 
            PulseLen, Prf)
    
Nesz_dB_Arr = 10*np.log10(NESZ_Arr)

In [ ]:
# plot the NESZ
fig, ax1 = plt.subplots(figsize = (8, 4))
# secondary axis for ground range values
ax2 = ax1.twiny()
# Add some extra space for the second axis at the bottom
fig.subplots_adjust(bottom=0.2)

# locations on the first axis, where the second axis values are calculated
new_tick_locations = np.array([32, 34, 36, 38, 40])

# convert between the one axis to the next
def tick_function(altitude, x):
    V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
    return ["%.0f" % z for z in V / 1000]

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.2))
# sett the ticks values and locations on the second axis
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
ax2.set_xlabel("Ground Range (km)")

for i in range(NumRxScanAngles * NumTxScanAngles):
    if i == 0:
        ax1.plot(Utils.rad2deg(IA_Arr[i]), Nesz_dB_Arr[i], color="red", ls="--", label="Calculated NESZ")
    else:
        ax1.plot(Utils.rad2deg(IA_Arr[i]), Nesz_dB_Arr[i], color="red", ls="--")
ax1.plot(Utils.rad2deg(Nesz_max_incidence_angles/1000), 10*np.log10(Nesz_max_values), color="green", label=r"Effective NESZ")

# plot required NESZ value +- 2dB
ax1.axhline(ReqNesz, color="blue", label="Required NESZ")
ax1.fill_between(ax1.get_xlim(), ReqNesz-2, ReqNesz+2, color="yellow", alpha=0.3)

# plot the desired incidence angles limits (based on PRF analysis)
iaMax = IncidenceAngleRange.max()
iaMin = IncidenceAngleRange.min()

ax1.axvline(x=iaMin, linestyle="dashed", color="blue")
ax1.axvline(x=iaMax, linestyle="dashed", color="blue")

# ax1.set_title("NESZ vs Incidence Angle")
ax1.set_xlabel(r"Incidence Angle (deg)")
ax1.set_ylabel(r"NESZ (dB)")
ax1.grid(linestyle='dotted')
ax1.legend(loc=4, framealpha=1.)
ax1.set_ylim(-36, -22)
ax1.margins(0)
ax2.set_xlim(ax1.get_xlim())

# plt.show()
if SAVE_FIGURES:
    if SAVE_FIGURE_FORMAT == "SVG":
        plt.savefig('../figures/fig_16_refl_verification_01_nesz.svg', format='svg', bbox_inches='tight')  
    elif SAVE_FIGURE_FORMAT == "PDF":
        plt.savefig('../figures/fig_16_refl_verification_01_nesz.pdf', format='pdf', bbox_inches='tight')  
    else:
        plt.savefig('../figures/fig_16_refl_verification_01_nesz.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
else:
    plt.show()

#### AASR

In [ ]:
# Calculate the antenna patterns for AASR calculationns
NumAmbiguities = 10
FrequencySteps = 100

# Get the processed Doppler bandwidth
DopplerBandwidth = [SEM.ProcessedDopplerBandwidth(SEM.PlatformVelocity(Altitude), ReqResolutionAz), ProcessedAzimuthBandwidth]
print("Processed Doppler Bandwidth: {:.2f} Hz".format(DopplerBandwidth[0]))
print("Processed Doppler Bandwidth: {:.2f} Hz".format(DopplerBandwidth[1]))

# DopplerFrequencies = np.linspace(-DopplerBandwidth/2, DopplerBandwidth/2, FrequencySteps)
DopplerFrequencies = np.linspace(start=-10000, stop=10000, num=1000)

AzimuthAngles = SEM.AzimuthAngleFromDoppler(DopplerFrequencies, Wavelength, SEM.PlatformVelocity(Altitude))
                        
# calculate the TX/RX antenna patterns over the Doppler frequency bandwidth
txAzPat = txAnt.azimuth_field_pattern(Wavelength, AzimuthAngles, 0)
rxAzPat = rxAnt.azimuth_field_pattern(Wavelength, AzimuthAngles, 0)

txAzPowerPat = txAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0)
rxAzPowerPat = rxAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0)

In [ ]:
# Plot the antenna patterns
fig, ax1 = plt.subplots(figsize = (12,3))
gainScale = tandemL_TX.gain(Wavelength) / RX_gain_full
# TX beam
ax1.plot(Utils.rad2deg(AzimuthAngles), 10*np.log10(txAzPowerPat * gainScale), color="blue", label=r"TX")
# RX beams
ax1.plot(Utils.rad2deg(AzimuthAngles), 10*np.log10(rxAzPowerPat), color="green", linestyle="dashed", label=r"RX")

ax1.set_title("Noramlised 3dB Antenna azimuth power pattern vs Angle")
ax1.set_xlabel(r"Angle (deg)")
ax1.set_ylabel(r"Antenna pattern (dB)")
ax1.grid(linestyle='dotted')
ax1.set_ylim(-40, 5)
ax1.set_xlim(-2, 2)
ax1.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the AASR values
if is_main_module():
    Aasr_Arr = [
        AASR.Get_AASR(
            DopplerBandwidth = DopplerBandwidth[0],
            Frequencies = DopplerFrequencies,
            NumAmbiguities = NumAmbiguities,
            FreqSteps = FrequencySteps,
            Prf = Prf,
            TxAntennaPattern = txAzPat,
            RxAntennaPattern = rxAzPat,
            TxGain = txAnt.gain(Wavelength),
            RxGain = rxAnt.gain(Wavelength) ),
                
            AASR.Get_AASR(
                DopplerBandwidth = DopplerBandwidth[1],
                Frequencies = DopplerFrequencies,
                NumAmbiguities = NumAmbiguities,
                FreqSteps = FrequencySteps,
                Prf = Prf,
                TxAntennaPattern = txAzPat,
                RxAntennaPattern = rxAzPat,
                TxGain = txAnt.gain(Wavelength),
                RxGain = rxAnt.gain(Wavelength) ) ]

    print("AASR = {:.2f} dB with Doppler bandwidth = {:.2f} Hz".format(10*np.log10(Aasr_Arr[0]), DopplerBandwidth[0]))
    print("AASR = {:.2f} dB with Doppler bandwidth = {:.2f} Hz".format(10*np.log10(Aasr_Arr[1]), DopplerBandwidth[1]))
    # create dummy gincidence angle array over which to plot the AASR data
    SwathIaArr = np.linspace(np.asarray(IA_Arr[0]).min(), np.asarray(IA_Arr[len(IA_Arr)-1]).max(), 50)
    
    # create an array equal to the incidence angle array and fill with the AASR data    
    Aasr_Arr_Full = [np.full((SwathIaArr.shape[0]), 10*np.log10(Aasr_Arr[0])),
                     np.full((SwathIaArr.shape[0]), 10*np.log10(Aasr_Arr[1]))]

In [ ]:
# plot the AASR vs ground swath
if is_main_module():
    fig, ax1 = plt.subplots(1,1, figsize = (8,4))    

    # secondary axis for ground range values
    ax2 = ax1.twiny()
    # Add some extra space for the second axis at the bottom
    fig.subplots_adjust(bottom=0.2)

    # locations on the first axis, where the second axis values are calculated
    new_tick_locations = np.array([32, 33, 34, 35, 36, 37, 38, 39, 40])

    # convert between the one axis to the next
    def tick_function(altitude, x):
        V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
        return ["%.0f" % z for z in V / 1000]

    # Move twinned axis ticks and label from top to bottom
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")

    # Offset the twin axis below the host
    ax2.spines["bottom"].set_position(("axes", -0.2))
    # sett the ticks values and locations on the second axis
    ax2.set_xticks(new_tick_locations)
    ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
    ax2.set_xlabel("Ground Range (km)")

    ax1.plot(np.rad2deg(SwathIaArr), Aasr_Arr_Full[0], color='tab:red', label=r'Calculated AASR with $B_\gamma$ = {:.2f} Hz'.format(DopplerBandwidth[0]))  
    ax1.plot(np.rad2deg(SwathIaArr), Aasr_Arr_Full[1], color='tab:orange', label=r'Calculated AASR with $B_\gamma$ = {:.2f} Hz'.format(DopplerBandwidth[1]))  
    
    ax1.axhline(y=ReqAasr, label="Required AASR", color="blue", linestyle="dashed")
    
    # ax1.set_title('Azimuth-Ambiguity-to-Signal Ratio vs Slant Range')
    ax1.set_ylabel(r'AASR (dB)')
    ax1.set_xlabel(r'Incidence Angle (deg)')
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.set_ylim(-30, -17)
    ax1.margins(0)
    ax1.legend(framealpha=1.)
    ax2.set_xlim(ax1.get_xlim())
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_17_refl_verification_01_aasr.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_17_refl_verification_01_aasr.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_17_refl_verification_01_aasr.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

#### RASR

In [ ]:
# Calculate the antenna patters and scan angles for the main RASR beams
if is_main_module():
    NumSamples = 100
    NumRxScanAngles = 31
    NumTxScanAngles = 1
    NF = 5
    NN = 3
    prf = Prf

    #------------------------------------------------------------
    # firstly get the antenna patterns and anges for the main antenna beams
    RxAntPatterns, TxAntPatterns, \
    rxtxAnglesRad, rxPeaks, \
    RxAntPatMaxAngles, RxAntPatMaxValues \
        = RASR.CalRasrMainAntennaPatterns_rect(
            rxAnt, 
            txAnt, 
            Wavelength, 
            NumRxScanAngles, 
            NumTxScanAngles,
            LookAngleRangeRad.min(), 
            LookAngleRangeRad.max(),
            NumSamples)

In [ ]:
# Plot the main antenna patterns
if is_main_module():
    fig, ax1 = plt.subplots(figsize = (8,3))
    for i in range(NumRxScanAngles * NumTxScanAngles):
        if i == 0:
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + Utils.deg2rad(AntennaOffset)), 10*np.log10(TxAntPatterns[i]), color="red", label="TX Elevation")
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + Utils.deg2rad(AntennaOffset)), 10*np.log10(RxAntPatterns[i]), color="black", linestyle="dashed", label="RX Elevation")
        else:
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + Utils.deg2rad(AntennaOffset)), 10*np.log10(TxAntPatterns[i]), color="red")
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + Utils.deg2rad(AntennaOffset)), 10*np.log10(RxAntPatterns[i]), color="black", linestyle="dashed")

    ax1.set_title("3dB Antenna pattern vs Angle")
    ax1.set_xlabel(r"Angle (deg)")
    ax1.set_ylabel(r"Antenna pattern (dB)")
    ax1.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.legend()
    plt.show()

In [ ]:
# calculate the incidence angles and slant ranges over the antenna beam pattern
if is_main_module():
    IA_Arr, SR_Arr \
        = RASR.CalcAnglesAndSlantRanges(
            NumRxScanAngles * NumTxScanAngles, 
            Altitude, 
            rxtxAnglesRad, 
            Utils.deg2rad(AntennaOffset))

In [ ]:
# calculate the main RASR values for each slant range
if is_main_module():
    RasrMain = RASR.Get_RasrMain(
        SlantRange = SR_Arr,
        IncidenceAngle = IA_Arr,
        TxAntennaPattern = np.asarray(TxAntPatterns), 
        RxAntennaPattern = np.asarray(RxAntPatterns) )
    RasrMain_dB = 10*np.log10(RasrMain)

In [ ]:
# Plot RASR Main values
if is_main_module():
    fig, ax1 = plt.subplots(figsize = (8, 3))
    for i in range(NumRxScanAngles * NumTxScanAngles):
        if i == 0:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), RasrMain_dB[i], color="red", ls="--", label="RASR")
        else:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), RasrMain_dB[i], color="red", ls="--")

    ax1.axhline(ReqNesz, color="blue", label="Required RASR")
    ax1.set_title("RASR vs Slant Range")
    ax1.set_xlabel(r"Incidence Angle (deg)")
    ax1.set_ylabel(r"RASR (dB)")
    ax1.grid(linestyle='dotted')
    ax1.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Calculate the slant ranges, incidence angles, and RX/TX antenna patterns over the ambiguous PRFs
if is_main_module():
    TX_Amb_Arr, RX_Amb_Arr, SR_Amb_Arr, IA_Amb_Arr \
        = RASR.CalcAmbiguousAntennaPatterns_rect(
            rxAnt, 
            txAnt, 
            NumRxScanAngles * NumTxScanAngles,
            SR_Arr, 
            NN, NF, 
            Altitude, 
            AntennaOffset, 
            Wavelength, 
            prf)

In [ ]:
# for each slant range and each ambiguity count, calculate the ambiguous value
if is_main_module():
    Rasr_Amb_Scan_Arr = []
    for i in range(NumRxScanAngles * NumTxScanAngles):
        Rasr_Amb_Arr = []
        for j in range(SR_Amb_Arr[i].shape[0]):
            Rasr_Amb_Arr.append(
                RASR.RasrAmb(
                    TX_Amb_Arr[i][j], 
                    RX_Amb_Arr[i][j], 
                    SR_Amb_Arr[i][j], 
                    IA_Amb_Arr[i][j]) )
        Rasr_Amb_Scan_Arr.append(Rasr_Amb_Arr)
    Rasr_Amb_Scan_Arr = np.asarray(Rasr_Amb_Scan_Arr)

In [ ]:
# add all the ambiguities for a given slant range together,
# this is the total ambiguity singal for a given slant range
if is_main_module():
    Rasr_Amb_sum = []

    for i in range(NumRxScanAngles * NumTxScanAngles):
        Rasr_Amb_sum.append(np.nansum(Rasr_Amb_Scan_Arr[i], axis=0))
            
    Rasr_Amb_sum = np.asarray(Rasr_Amb_sum)

    # ----------------------------------------------------------
    # calculate the Total RASR value for each slant range
    RASR_Total_Arr = [] = []
    for i in range(NumRxScanAngles * NumTxScanAngles):
        RASR_Arr = []
        for j in range(SR_Arr[i].shape[0]):
            RASR_Arr.append(RasrMain[i][j] * Rasr_Amb_sum[i][j])
        RASR_Total_Arr.append(RASR_Arr)

    RASR_Total_Arr = np.asarray(RASR_Total_Arr)
    RASR_Arr_dB = 10*np.log10(RASR_Total_Arr)

In [ ]:
# Plot RASR values vs ground
if is_main_module():
    fig, ax1 = plt.subplots(figsize = (8, 4))

    # secondary axis for ground range values
    ax2 = ax1.twiny()
    # Add some extra space for the second axis at the bottom
    fig.subplots_adjust(bottom=0.2)

    # locations on the first axis, where the second axis values are calculated
    new_tick_locations = np.array([32, 33, 34, 35, 36, 38, 37, 39, 40])

    # convert between the one axis to the next
    def tick_function(altitude, x):
        V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
        return ["%.0f" % z for z in V / 1000]

    # Move twinned axis ticks and label from top to bottom
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")

    # Offset the twin axis below the host
    ax2.spines["bottom"].set_position(("axes", -0.2))
    # sett the ticks values and locations on the second axis
    ax2.set_xticks(new_tick_locations)
    ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
    ax2.set_xlabel("Ground Range (km)")

    for i in range(NumRxScanAngles * NumTxScanAngles):
        if i == 0:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), RASR_Arr_dB[i], color="red", ls="--", label="RASR")
        else:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), RASR_Arr_dB[i], color="red", ls="--")

    ax1.axhline(ReqRasr, color="blue", label="Required RASR")
    
    ax1.axvline(x=Utils.rad2deg(IncidenceAngleRangeRad.min()), color="blue", linestyle="dashed")
    ax1.axvline(x=Utils.rad2deg(IncidenceAngleRangeRad.max()), color="blue", linestyle="dashed")

    # ax1.set_title("RASR vs Slant Range")
    ax1.set_xlabel(r"Incidence Angle (deg)")
    ax1.set_ylabel(r"RASR (dB)")
    ax1.grid(linestyle='dotted')
    ax1.legend(framealpha=1.)
    ax1.set_ylim(-60, -17)
    ax1.set_xlim(31.5, 41)
    ax1.margins(0)
    ax2.set_xlim(ax1.get_xlim())
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_18_refl_verification_01_rasr.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_18_refl_verification_01_rasr.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_18_refl_verification_01_rasr.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()